In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%%capture
!pip install bertopic
!pip install -U kaleido

In [3]:
import re
import pandas as pd
from datetime import datetime



In [4]:
# Load data
ny_time = pd.read_csv('/content/gdrive/MyDrive/Cendie/ny_times(1).csv')


In [5]:
ny_times = ny_time.dropna(subset = ["Titulo"]).reset_index(drop=True)

In [6]:
ny_times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  978 non-null    int64 
 1   Titulo      978 non-null    object
 2   Url         978 non-null    object
 3   Fecha       978 non-null    object
 4   Texto       978 non-null    object
dtypes: int64(1), object(4)
memory usage: 38.3+ KB


In [7]:
ny_times.Texto = ny_times.apply(lambda row: re.sub(r"http\S+", "", row.Texto).lower(), 1)
ny_times.Texto= ny_times.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.Texto.split())), 1)
ny_times.Texto = ny_times.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.Texto).split()), 1)
ny_times_Texto = ny_times.Texto.to_list()

In [ ]:
# nypost_texto= []
# for i in range(len(ny_post_texto)):
#   a = ny_post_texto[i].replace("sign up for our special edition newsletter to get a daily update on the coronavirus pandemic ", "") 
#   nypost_texto.append(a)

In [8]:
len(ny_times_Texto)

978

In [9]:
timestamps = ny_times.Fecha.to_list()
len(timestamps)


978

In [10]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

#text = "Nick likes to play football, however he is not too fond of tennis."
# nytimes_text= []

# for i in range(len(ny_times_Texto)):
#   text_tokens = word_tokenize(ny_times_Texto[i])
#   nytimes_text.append(text_tokens)

# #tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]

# print(nytimes_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
nytimes_texto= []

for i in range(len(ny_times_Texto)):
  #text_tokens = [word for word in nytimes_text[i] if not word in stopwords.words()]
  #nytimes_texto.append(text_tokens)
  pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
  text = pattern.sub('', ny_times_Texto[i])
  nytimes_texto.append(text)



In [12]:
len(nytimes_texto)

978

In [13]:
nytimes_text = pd.Series(nytimes_texto)
 


In [14]:
len(nytimes_text)

978

In [ ]:
#nytimes_text

In [15]:
timestamps = pd.Series(timestamps)

In [16]:
#timestamps.dtype
#timestamps.date

#pd.to_datetime(timestamps, format='%Y%m%d')

#timestamps.dt.date

timestamps= pd.to_datetime(timestamps, errors='coerce', utc=True)
timestamps = timestamps.dt.date


In [30]:
timestamps

0      2020-01-21
1      2020-03-09
2      2020-01-26
3      2020-02-10
4      2020-07-03
          ...    
973    2020-10-25
974    2020-10-25
975    2020-10-25
976    2020-10-29
977    2020-11-02
Length: 978, dtype: object

In [44]:
df_combinados = pd.concat([nytimes_text, timestamps], axis = 1)
df_combinados = df_combinados.rename(columns={0: "Texto", 1: "Fecha"})
df_combinados

,Texto,Fecha
0,beijing authorities china reported third death...,2020-01-21
1,briefing ended read developments live coronavi...,2020-03-09
2,united states confirmed third case novel coron...,2020-01-26
3,beijing took thousands infections scores death...,2020-02-10
4,wuhan china top chinese health official warned...,2020-07-03
...,...,...
973,nine days election day president trump joseph ...,2020-10-25
974,washington virtual conference september adam b...,2020-10-25
975,thursday morning lincoln project anti trump gr...,2020-10-25
976,green bay wis town lives dies packers metaphor...,2020-10-29


In [68]:
import datetime
startdate = datetime.datetime.strptime('2020-03-01', "%Y-%m-%d").date()
enddate = datetime.datetime.strptime('2020-10-31', "%Y-%m-%d").date()
mask = (df_combinados['Fecha'] >= startdate) & (df_combinados['Fecha'] <= enddate)
df_combinados1 = df_combinados.loc[mask]
df_combinados1 = df_combinados1.reset_index()
df_combinados1

,index,Texto,Fecha
0,1,briefing ended read developments live coronavi...,2020-03-09
1,4,wuhan china top chinese health official warned...,2020-07-03
2,5,article part developing coronavirus coverage m...,2020-03-09
3,6,briefing ended read latest developments corona...,2020-03-12
4,7,bangkok myanmar loudspeakers broadcast advice ...,2020-09-25
...,...,...,...
870,972,first requests upended seesaw popular classroo...,2020-10-25
871,973,nine days election day president trump joseph ...,2020-10-25
872,974,washington virtual conference september adam b...,2020-10-25
873,975,thursday morning lincoln project anti trump gr...,2020-10-25


In [ ]:
#nytimes_text = nytimes_text.apply(lambda x: ', '.join([str(i) for i in x]))

In [ ]:
# import re
# regex_dict = {'punct': re.compile(r'[^@#\w_]')}
   

In [ ]:
#nytimes_text = nytimes_text.str.replace(regex_dict['punct'], ' ')

In [ ]:
#nytimes_text

In [70]:
from bertopic import BERTopic
topic_model = BERTopic(language="english", calculate_probabilities = True)# min_topic_size=10, nr_topics=10 , verbose=True)
topics, probs = topic_model.fit_transform(df_combinados1['Texto'])

In [71]:
freq = topic_model.get_topic_info(); freq.head(30)

,Topic,Count,Name
0,-1,356,-1_coronavirus_health_virus_pandemic
1,0,103,0_trump_president_virus_coronavirus
2,1,98,1_virus_coronavirus_china_health
3,2,40,2_cai_patients_hospital_care
4,3,39,3_music_theater_concert_show
5,4,37,4_workers_employees_company_restaurant
6,5,33,5_students_school_schools_children
7,6,27,6_season_league_sports_baseball
8,7,20,7_loan_cash_unemployment_loans
9,8,18,8_police_protesters_protests_mayor


In [72]:
for i in range(1, 18):    
    topic_nr = freq.iloc[i]["Topic"]
    print(f'Tópico {i-1}:')
    print(topic_model.get_topic(topic_nr)) 

Tópico 0:
[('trump', 0.03782682379164416), ('president', 0.02552811299885022), ('virus', 0.011933643871201046), ('coronavirus', 0.011503296869797935), ('biden', 0.01060094310215683), ('pandemic', 0.010248654891955809), ('states', 0.010182926270164835), ('campaign', 0.00959772453947834), ('republican', 0.008885256165679256), ('political', 0.008627871095683942)]
Tópico 1:
[('virus', 0.016940685127241114), ('coronavirus', 0.016591111718207888), ('china', 0.01637804668195501), ('health', 0.015264049823793573), ('chinese', 0.012250943799565666), ('tests', 0.009103893168161684), ('disease', 0.008916650115155278), ('testing', 0.008901489052082944), ('dr', 0.00888644768823696), ('pandemic', 0.008852758472274673)]
Tópico 2:
[('cai', 0.018887135955926494), ('patients', 0.016798458057774582), ('hospital', 0.01658041632419387), ('care', 0.013999217634278653), ('died', 0.01286177747482207), ('covid', 0.01253356363245796), ('family', 0.012288715811587348), ('coronavirus', 0.0118088981632856), ('dr',

In [73]:
fig = topic_model.visualize_hierarchy()
fig.update_layout(title_text = 'Dendograma - Ny time')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_times/Dendograma.jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_times/Dendograma.html")

In [74]:
fig = topic_model.visualize_barchart([0,1,2,3,4,5], n_words = 10)
fig.update_layout(title_text = 'Tópicos - Ny time')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_times/score-topics(1).jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_times/score-topics(1).html")
fig

In [75]:
fig = topic_model.visualize_barchart([6,7,8,9,10,11], n_words = 10)
fig.update_layout(title_text = '')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_times/score-topics(2).jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_times/score-topics(2).html")
fig

In [78]:
fig = topic_model.visualize_barchart([12,13, 14, 15, 16], n_words = 10)
fig.update_layout(title_text = '')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_times/score-topics(3).jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_times/score-topics(3).html")
fig

In [79]:
fig = topic_model.visualize_heatmap()
fig.update_layout(title_text = 'Matriz de correlación')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_times/correlación.jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_times/correlación.html")
fig

In [80]:
topic_model.visualize_term_rank()

In [81]:
fig = topic_model.visualize_topics(); fig

In [83]:
topics_over_time = topic_model.topics_over_time(docs=df_combinados1['Texto'], 
                                                topics=topics, 
                                                timestamps=df_combinados1['Fecha'], 
                                                global_tuning=True,
                                                evolution_tuning=True)
                                                #nr_bins=20)

In [86]:
fig = topic_model.visualize_topics_over_time(topics_over_time)
fig.update_layout(title_text = 'Tópicos a través del tiempo - Ny Times')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_times/topics-overtime.jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_times/topics-overtime.html")
fig

In [85]:
probabilidades = []
for i in range(0, 978):
  proba = pd.Series(probs[i].argmax())
  proba = proba[0]
  probabilidades.append(proba)

print(probabilidades)
  


IndexError: ignored

In [ ]:
probabilidades[977]

0

In [ ]:
probabilidades = pd.Series(probabilidades, name = 'Topico más probable')


In [ ]:
ny_times_contopicos = pd.concat([ny_times, probabilidades], axis = 1)
ny_times_contopicos =  ny_times_contopicos[[ 'Titulo', 'Url', 'Fecha' ,'Texto' ,'Topico más probable']]
ny_times_contopicos

,Titulo,Url,Fecha,Texto,Topico más probable
0,Deadly Mystery Virus Reported in 2 New Chinese...,https://www.nytimes.com/2020/01/18/world/asia/...,2020-01-21T14:39:21-05:00,beijing the authorities in china reported a th...,4
1,"More U.S. Coronavirus Cases Emerge, as China’s...",https://www.nytimes.com/2020/01/26/world/china...,2020-03-09T12:22:25-04:00,this briefing has ended read about development...,4
2,"California Confirms Coronavirus Patient, Marki...",https://www.nytimes.com/2020/01/26/world/asia/...,2020-01-26T04:14:04-05:00,the united states has confirmed a third case o...,4
3,"In Coronavirus, a ‘Battle’ That Could Humble C...",https://www.nytimes.com/2020/01/26/world/asia/...,2020-02-10T12:05:50-05:00,beijing it took thousands of infections and sc...,4
4,"As Coronavirus Fears Intensify, Effectiveness ...",https://www.nytimes.com/2020/01/26/world/asia/...,2020-07-03T18:55:10-04:00,wuhan china a top chinese health official warn...,4
...,...,...,...,...,...
973,With a little more than a week until Election ...,https://www.nytimes.com/2020/10/25/us/election...,2020-10-25T16:44:30-04:00,nine days before election day president trump ...,0
974,Kodak Loan Debacle Puts a New Agency in the Ho...,https://www.nytimes.com/2020/10/25/business/ec...,2020-10-25T17:05:00-04:00,washington at a virtual conference in septembe...,0
975,N.Y.C. billboards pair Ivanka Trump and Jared ...,https://www.nytimes.com/2020/10/25/world/nyc-b...,2020-10-25T17:09:26-04:00,on thursday morning the lincoln project an ant...,0
976,"Game Day in Green Bay: Empty Sports Bars, Full...",https://www.nytimes.com/2020/10/25/us/wisconsi...,2020-10-29T13:03:57-04:00,green bay wis in a town that lives or dies wit...,12
